## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-23-02-07-07 +0000,nypost,Copenhagen Airport suspends flights due to lar...,https://nypost.com/2025/09/22/world-news/copen...
1,2025-09-23-02-06-09 +0000,bbc,Copenhagen Airport reopens after drone sightin...,https://www.bbc.com/news/articles/cn4lj1yvgvgo...
2,2025-09-23-02-03-25 +0000,nyt,‘I Hate My Opponent’: Trump’s Remarks at Kirk ...,https://www.nytimes.com/2025/09/22/us/politics...
3,2025-09-23-02-00-38 +0000,bbc,Trump urges pregnant women to avoid Tylenol ov...,https://www.bbc.com/news/articles/cx20d4lr67lo...
4,2025-09-23-01-55-09 +0000,nypost,Republicans and NJ gov. candidate Jack Ciattar...,https://nypost.com/2025/09/22/us-news/republic...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,63
16,kirk,27
111,kimmel,20
47,charlie,20
102,palestinian,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
185,2025-09-22-18-27-02 +0000,nypost,"Elon Musk, President Trump come together at Ch...",https://nypost.com/2025/09/22/us-news/elon-mus...,150
335,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...,138
283,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...,137
154,2025-09-22-19-59-40 +0000,nypost,Trump open to national holiday honoring Charli...,https://nypost.com/2025/09/22/us-news/trump-li...,131
279,2025-09-22-11-42-53 +0000,bbc,Trump hails Charlie Kirk as martyr to thousand...,https://www.bbc.com/news/articles/ckgee0x9p40o...,125


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
185,150,2025-09-22-18-27-02 +0000,nypost,"Elon Musk, President Trump come together at Ch...",https://nypost.com/2025/09/22/us-news/elon-mus...
232,78,2025-09-22-15-31-23 +0000,nypost,Howard Stern blasts ABC suspension of Jimmy Ki...,https://nypost.com/2025/09/22/media/howard-ste...
112,50,2025-09-22-21-34-40 +0000,nypost,Trump announces FDA will change Tylenol safety...,https://nypost.com/2025/09/22/us-news/trump-an...
300,46,2025-09-22-10-00-00 +0000,nypost,NJ governor’s race heats up as Ciattarelli’s G...,https://nypost.com/2025/09/22/us-news/the-kama...
272,45,2025-09-22-12-55-13 +0000,nypost,Putin says he is open to one-year extension of...,https://nypost.com/2025/09/22/world-news/putin...
284,43,2025-09-22-11-11-45 +0000,bbc,UK says Israel must not retaliate against Pale...,https://www.bbc.com/news/articles/c1wggrdn9dno...
183,35,2025-09-22-18-28-40 +0000,nypost,Russia risks ‘expansion’ of war with intrusion...,https://nypost.com/2025/09/22/us-news/russia-r...
38,32,2025-09-23-00-34-00 +0000,wsj,The team of federal economists and researchers...,https://www.wsj.com/politics/policy/usda-puts-...
268,31,2025-09-22-13-00-00 +0000,wsj,Atlanta Fed President Raphael Bostic said infl...,https://www.wsj.com/economy/central-banking/fe...
50,30,2025-09-23-00-03-00 +0000,wsj,The bipartisan organization overseeing the nat...,https://www.wsj.com/politics/policy/group-orga...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
